In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config
from timeit import timeit





In [ ]:
##
##
## Full pipeline for extracting ONLY ring
##
##
##


# Event extraction
run_number = 28
event_number = 0
tel_number = 1
filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{run_number}.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)
for i,j in enumerate(event_iterator):
    if i == event_number:
        event = j
        break



# Building appropriate image for this event
plt.figure(figsize = (9,6))
camgeom = source.subarray.tel[1].camera.geometry
title="Processed Event"
disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True)
disp.image = event.simulation.tel[tel_number].true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)



In [ ]:
# Building appropriate image for this event
#plt.figure(figsize = (9,6))
#camgeom = source.subarray.tel[1].camera.geometry
#title="Processed Event"
#disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True)
#disp.image = event.r1.tel[tel_number].waveform[:,20]
#disp.image = proton_arr
#disp.image = event.simulation.tel[tel_number].true_image
#disp.cmap = plt.cm.RdBu_r
#disp.add_colorbar()
#disp.set_limits_percent(95)
#plt.savefig(f"/Users/vdk/ProcessedEvent_event{j.index.event_id}")



ring_arr = []
# If its not a ring we assign value of 100 to it, ring became visually distinguishable on 11 sample
working_arr = event.r1.tel[tel_number].waveform[:,12].copy()
for i,j in enumerate(working_arr):
    if working_arr[i] > 1 and working_arr[i] < 60:
        ring_arr.append(100)
    else:
        ring_arr.append(working_arr[i])


proton_arr = []
working_arr = event.r1.tel[tel_number].waveform[:,22].copy()
for i,j in enumerate(working_arr):
    if working_arr[i] > 1 and working_arr[i] < 50:
        proton_arr.append(100)
    else:
        proton_arr.append(working_arr[i])
        
        
    

In [ ]:
event.r0.tel[tel_number].waveform[1].shape

In [ ]:
pixel_mask = []
for i,j in enumerate(event.r0.tel[tel_number].waveform[1]):
    if np.argmax(j) == 12 or np.argmax(j) == 13 or np.argmax(j) == 14 or np.argmax(j) == 15: #or np.argmax(j) == 16: 
        pixel_mask.append(True)
    else:
        pixel_mask.append(False)

In [ ]:
image_arr = []

working_wave = event.r1.tel[tel_number].waveform.copy()

for i,j in enumerate(working_wave):
    if not pixel_mask[i]:
        working_wave[i] = np.zeros(40)     
        
        
# Building appropriate image for this event
plt.figure(figsize = (9,6))
camgeom = source.subarray.tel[1].camera.geometry
title="Extracted Mask"
disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True)
#disp.image = event.r1.tel[teln].waveform[:,11]
disp.image = working_wave.sum(axis=1)
#disp.image = event.simulation.tel[tel_number].true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
#plt.savefig(f"/Users/vdk/ExtractedMask_event{run_number}_{str(0)}_{event_number}")

In [ ]:
pixel_mask

In [ ]:
image_arr = []

working_wave = event.r1.tel[tel_number].waveform.copy()

for i,j in enumerate(working_wave):
    if not pixel_mask[i]:
        working_wave[i] = np.zeros(40)     
        
        
# Building appropriate image for this event
plt.figure(figsize = (9,6))
camgeom = source.subarray.tel[1].camera.geometry
title="Extracted Mask"
disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True)
#disp.image = event.r1.tel[teln].waveform[:,11]
disp.image = working_wave.sum(axis=1)
#disp.image = event.simulation.tel[tel_number].true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
#plt.savefig(f"/Users/vdk/ExtractedMask_event{run_number}_{str(0)}_{event_number}")

In [ ]:
working_wave = event.r1.tel[tel_number].waveform.copy()

for i,j in enumerate(working_wave):
    if not pixel_mask[i]:
        working_wave[i] = np.zeros(40)
        
for i in working_wave:
    print(i)

In [ ]:
len(image_arr[:,0])

In [ ]:
for i in image_arr.sum(axis=1):
    print(i)

In [ ]:
image_arr = []

working_arr = event.r0.tel[tel_number].waveform[1].copy()
for i,j in enumerate(working_arr):
    if i in pixel_mask:
        image_arr.append(j)
    else:
        image_arr.append(np.zeros(40))
        
image_arr = np.array(image_arr)       
        
        
# Building appropriate image for this event
plt.figure(figsize = (9,6))
camgeom = source.subarray.tel[1].camera.geometry
title="Extracted Mask"
disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True)
disp.image = event.r1.tel[tel_number].waveform.sum(axis=1)
#disp.image = image_arr.sum(axis=1)
#disp.image = event.simulation.tel[tel_number].true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
#plt.savefig(f"/Users/vdk/ExtractedMask_event{run_number}_{str(0)}_{event_number}")

In [ ]:
#
#
#
#
#
# Main algorihtm to process events with mask to extract ONLY muon ring
#
#
#
#
#


#2223, 9222, 2438, 25051, 14406, 22126, 22365, 32298, 35219, 8234, 8448, 9404, 9433, 27374, 28328, 28000, 35298
contaminated_runs = [2,9,2,25,14,22,22, 32,35,8,8,9,9,27,28,28,35]
contaminated_events = [223, 222, 438, 51, 406, 126, 365, 298, 219, 234, 448, 404, 433, 374, 328, 0, 298]
tel_numbers = [2, 2,3,1,1,1,1, 3, 2, 1,1,1,3,2,1,1,2]
events = []
working_waves = []

for i,j in enumerate(contaminated_runs):
    tel_number = tel_numbers[i]
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{j}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    for k,l in enumerate(event_iterator):
        if k == contaminated_events[i]:
            event = l
            events.append(event)
            break
    # Building appropriate image for this event
    
    
    fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
    fig.subplots_adjust(hspace=0.2)
    ax = axes[0]
    camgeom = source.subarray.tel[1].camera.geometry
    title= f"True Image of Event_{str(contaminated_runs[i])+'run'+str(contaminated_events[i])}"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax = ax)
    disp.image = event.simulation.tel[tel_number].true_image
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    bx = axes[1]
    pixel_mask = []
    for k,l in enumerate(event.r0.tel[tel_number].waveform[1]):
        if np.argmax(l) in [13,14,15]: 
            pixel_mask.append(True)
        else:
            pixel_mask.append(False)
    working_wave = event.r1.tel[tel_number].waveform.copy()
    for k,l in enumerate(working_wave):
        if not pixel_mask[k]:
            working_wave[k] = np.zeros(40)     
            #working_wave[k] = np.min(event.r1.tel[tel_number].waveform, axis = 0)
    working_waves.append(working_wave)
    event.r1.tel[tel_numbers[i]].waveform = working_wave
    image_processor = ImageProcessor(source.subarray)
    muon_processor = MuonProcessor(source.subarray)
    calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
    calib(event)
    image_processor(event)
    muon_processor(event)
    camgeom = source.subarray.tel[1].camera.geometry
    title= f"Extracted by mask. fitted optical eff = {round(event.muon.tel[tel_number].efficiency.optical_efficiency, 3)}"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax=bx)
    disp.image = working_wave.sum(axis=1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    #plt.savefig(f"/Users/vdk/ProcessedEventsWithMask_event{str(contaminated_runs[i])+str(contaminated_events[i])}")

    
plt.close()



In [ ]:
event_iterator

In [ ]:
dir(source)

In [ ]:
print(source.metadata)

In [ ]:
len(events)

In [ ]:
np.min(event.r1.tel[tel_number].waveform, axis = 0).shape

In [ ]:
working_waves = np.array(working_waves)
opt_eff = []
for i,event in enumerate(events):
    # Firstly without mask
    event.r1.tel[tel_numbers[i]].waveform = working_waves[i]
    image_processor = ImageProcessor(source.subarray)
    muon_processor = MuonProcessor(source.subarray)
    calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
    calib(event)
    image_processor(event)
    muon_processor(event)
    print("COG = ",  event.dl1.tel[1].parameters.concentration.cog)
    #if not np.isnan(event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency) and event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency < 0.25:
    if not np.isnan(event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency):
        opt_eff.append(event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency)
    print(f"Event number {str(contaminated_runs[i])+str(contaminated_events[i])}")
    print("With Mask", event.muon.tel[tel_numbers[i]].efficiency)

    
    
    

In [ ]:
opt_eff

In [ ]:
#2223, 9222, 2438, 25051, 14406, 22126, 22365, 32298, 35219, 8234, 8448, 9404, 9433, 27374, 28328, 28000, 35298
contaminated_runs = [2,9,2,25,14,22,22, 32,35,8,8,9,9,27,28,28,35]
contaminated_events = [223, 222, 438, 51, 406, 126, 365, 298, 219, 234, 448, 404, 433, 374, 328, 0, 298]
tel_numbers = [2, 2,3,1,1,1,1, 3, 2, 1,1,1,3,2,1,1,2]

for i,j in enumerate(contaminated_runs):
    tel_number = tel_numbers[i]
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{j}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    for k,l in enumerate(event_iterator):
        if k == contaminated_events[i]:
            event = l
            break
    image_processor = ImageProcessor(source.subarray)
    muon_processor = MuonProcessor(source.subarray)
    calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
    calib(event)
    image_processor(event)
    muon_processor(event)
    print(f"Event number {str(contaminated_runs[i])+str(contaminated_events[i])}")
    print("Without Mask", event.muon.tel[tel_numbers[i]].efficiency)     




In [ ]:
x_range = [i for i in range(len(opt_eff))]
#std_arr = [np.std(opt_eff)/np.mean(opt_eff) for i,j in enumerate(opt_eff)]
std_arr = [np.std(opt_eff) for i,j in enumerate(opt_eff)]
plt.xlabel("Event")
#plt.ylabel("Optical eff with rel. error for 1 measurment")
plt.ylabel("Optical eff with std for 1 measurment")
plt.grid(alpha=0.5)
#plt.ylim(0,0.45)
plt.scatter(x_range, opt_eff)
plt.title("Opt eff for muon rings extracted by mask")
plt.errorbar(x_range, opt_eff, yerr=std_arr, fmt='o', ms = 10)#, c = 'blue')#, s = 100)#, c= 'black')
plt.annotate("Ring not separated from shower", (3.2,opt_eff[3]), (6,0.335), arrowprops=dict(facecolor='red', shrink=0.05))
#plt.savefig("/Users/vdk/OptEffGraph")

In [ ]:
np.std(opt_eff)/np.mean(opt_eff)

In [ ]:
np.mean(opt_eff)

In [ ]:
np.std(opt_eff)

In [ ]:
np.std(opt_eff)/np.mean(opt_eff)

In [ ]:
###
###
### EXTRACT SHOWER FROM IMAGE
###
###
###


#
#
#
#
#
# Main algorihtm to process events with mask to extract ONLY muon ring
#
#
#
#
#

deleted_pixels = []

#2223, 9222, 2438, 25051, 14406, 22126, 22365, 32298, 35219, 8234, 8448, 9404, 9433, 27374, 28328, 28000, 35298
contaminated_runs = [2,9,2,25,14,22,22, 32,35,8,8,9,9,27,28,28,35]
contaminated_events = [223, 222, 438, 51, 406, 126, 365, 298, 219, 234, 448, 404, 433, 374, 328, 0, 298]
tel_numbers = [2, 2,3,1,1,1,1, 3, 2, 1,1,1,3,2,1,1,2]
events = []
working_waves = []


calib_config = Config({"GlobalPeakWindowSum": {"window_width" :7, "window_shift" : 3}})



for i,j in enumerate(contaminated_runs):
    counter = 0
    tel_number = tel_numbers[i]
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{j}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    for k,l in enumerate(event_iterator):
        if k == contaminated_events[i]:
            event = l
            events.append(event)
            break
    # Building appropriate image for this event
    
    
    fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
    fig.subplots_adjust(hspace=0.2)
    ax = axes[0]
    camgeom = source.subarray.tel[1].camera.geometry
    title= f"True Image of Event_{str(contaminated_runs[i])+'run'+str(contaminated_events[i])}"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax = ax)
    disp.image = event.simulation.tel[tel_number].true_image
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    bx = axes[1]
    pixel_mask = []
    time_bin_separation = 16
    for k,l in enumerate(event.r0.tel[tel_number].waveform[1]):
        if np.argmax(l) in range(time_bin_separation,40): 
            pixel_mask.append(True)
            counter += 1
        else:
            pixel_mask.append(False)
    working_wave = event.r1.tel[tel_number].waveform.copy()
    for k,l in enumerate(working_wave):
        if pixel_mask[k]:
            #working_wave[k] = np.zeros(40)     
            working_wave[k] = np.mean(event.r1.tel[tel_number].waveform, axis = 0)
    working_waves.append(working_wave)
    event.r1.tel[tel_numbers[i]].waveform = working_wave
    image_processor = ImageProcessor(source.subarray)
    muon_processor = MuonProcessor(source.subarray)
    calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray, config = calib_config)
    calib(event)
    image_processor(event)
    muon_processor(event)
    deleted_pixels.append(counter)
    #opt_effi = []
    #if not np.isnan(event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency):
    #opt_effi.append(event.muon.tel[tel_numbers[i]])
    camgeom = source.subarray.tel[1].camera.geometry
    title= f"Extracted by mask <{time_bin_separation}timebin, fitted optical eff = {round(event.muon.tel[tel_number].efficiency.optical_efficiency, 3)}"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax=bx)
    disp.image = working_wave.sum(axis=1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    #plt.show()
    #plt.savefig(f"/Users/vdk/ImagePurgatorium/ProcessedEvents<{time_bin_separation}timebinMEAN_event{str(contaminated_runs[i])+str(contaminated_events[i])}")
    
plt.close()


print("SHOWER PIXELS = ", counter)
print("RING PIXELS = ", len(event.r1.tel[tel_numbers[i]].waveform)-counter)

In [ ]:
working_waves = np.array(working_waves)
opt_eff = []
for i,event in enumerate(events):
    # Firstly without mask
    event.r1.tel[tel_numbers[i]].waveform = working_waves[i]
    #image_processor = ImageProcessor(source.subarray)
    #muon_processor = MuonProcessor(source.subarray)
    #calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
    #calib(event)
    #image_processor(event)
    #muon_processor(event)
    if i == 7:
        check_event = event
    #print("Params = ",  event.muon.tel[tel_numbers[i]])
    #print("Params = ",  abs(event.muon.tel[tel_numbers[i]].parameters.intensity_ratio-1))    
    #print("MSE = ", event.muon.tel[tel_numbers[i]].parameters.mean_squared_error, event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency)    
    #print("event = ", contaminated_runs[i],contaminated_events[i], "lets check", check_event.dl1.tel[tel_numbers[i]].parameters)
    #print("=========")
    #print("=========")
    #print("=========")
    print(event.dl1.tel[tel_numbers[i]].parameters.morphology.n_pixels)
    #if not np.isnan(event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency) and event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency < 0.25:
    if not np.isnan(event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency) and event.muon.tel[tel_numbers[i]].parameters.mean_squared_error.to_value() < 0.01:
        opt_eff.append(event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency)
    #print(f"Event number {str(contaminated_runs[i])+str(contaminated_events[i])}")
    #print("With Mask", event.muon.tel[tel_numbers[i]].efficiency)


    

In [ ]:
opt_eff
x_range = [i for i in range(len(opt_eff))]
#std_arr = [np.std(opt_eff)/np.mean(opt_eff) for i,j in enumerate(opt_eff)]
std_arr = [np.std(opt_eff) for i,j in enumerate(opt_eff)]
plt.xlabel("Event")
#plt.ylabel("Optical eff with rel. error for 1 measurment")
plt.ylabel("Optical eff with std for 1 measurment")
plt.grid(alpha=0.5)
#plt.ylim(0,0.45)
plt.scatter(x_range, opt_eff)
plt.title("Opt eff for muon rings extracted by mask")
plt.errorbar(x_range, opt_eff, yerr=std_arr, fmt='o', ms = 10)#, c = 'blue')#, s = 100)#, c= 'black')
#plt.annotate("Ring not separated from shower", (0.2,opt_eff[0]), (6,0.335), arrowprops=dict(facecolor='red', shrink=0.05))
#plt.savefig("/Users/vdk/OptEffGraph")

In [ ]:
np.mean(opt_eff)

In [ ]:
np.std(opt_eff)

In [ ]:
np.std(opt_eff)/np.mean(opt_eff)

In [ ]:
print(check_event.muon.tel[3])

In [ ]:
print(check_event.dl1.tel[3])

In [ ]:
max(check_event.dl1.tel[3].peak_time)

In [ ]:
run = 32
event_number = 298
tel_number = 3

counter = 0
deleted_pixels = []


filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{run}.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)
for k,l in enumerate(event_iterator):
    if k == event_number:
        event = l
        break

# Building appropriate image for this event
    
fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
fig.subplots_adjust(hspace=0.2)
ax = axes[0]
camgeom = source.subarray.tel[1].camera.geometry
title= f"True Image of Event_{str(run)+'run'+str(event_number)}"
disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax = ax)
disp.image = event.simulation.tel[tel_number].true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)

bx = axes[1]
pixel_mask = []
for k,l in enumerate(event.r0.tel[tel_number].waveform[1]):
    if np.argmax(l) in range(15,40): 
        pixel_mask.append(True)
        counter += 1
    else:
        pixel_mask.append(False)

working_wave = event.r1.tel[tel_number].waveform.copy()
for k,l in enumerate(working_wave):
    if pixel_mask[k]:
        #working_wave[k] = np.zeros(40)     
        working_wave[k] = np.mean(event.r1.tel[tel_number].waveform, axis = 0)


event.r1.tel[tel_number].waveform = working_wave

image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray)

calib(event)
image_processor(event)
muon_processor(event)
deleted_pixels.append(counter)
print("counter ", counter)
#print("deleted pixels ", deleted_pixels)
#opt_effi = []
#if not np.isnan(event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency):
#opt_effi.append(event.muon.tel[tel_numbers[i]])
camgeom = source.subarray.tel[1].camera.geometry
#title= f"Extracted by mask. fitted optical eff = {round(event.muon.tel[tel_number].efficiency.optical_efficiency, 3)}"
title= f"Extracted by mask r1 waveform"
disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax=bx)
disp.image = working_wave.sum(axis=1)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
#plt.show()
#plt.savefig(f"/Users/vdk/ProcessedEvents<16timebinMEANMEANMEAN_event{str(contaminated_runs[i])+str(contaminated_events[i])}")
plt.savefig(f"/Users/vdk/test_dpi.png", dpi=300)
   
plt.close()

In [ ]:
run = 4
event_number = 102
teln = 1
filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{run}.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)

for i,j in enumerate(event_iterator):
    if i == event_number:
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        event = j
        fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
        fig.subplots_adjust(hspace=0.2)
        ax = axes[0]
        camgeom = source.subarray.tel[1].camera.geometry
        title=f"event{event_number}_fit{round(j.muon.tel[teln].efficiency.optical_efficiency,3)}_width{round(j.muon.tel[teln].efficiency.width.to_value(),3)}"
        disp = CameraDisplay(camgeom,title=title, ax = ax)
        #disp.image = event.r1.tel[teln].waveform[:,11]
        #disp.image = event.dl0.tel[teln].waveform[:,11]
        disp.image = j.simulation.tel[teln].true_image #.sum(axis=1)
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        bx = axes[1]
        for pix_id in range(0,1855):
            #bx.plot(j.r0.tel[teln].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
            bx.plot(j.r1.tel[teln].waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')
        bx.set_ylabel("ADC counts")
        bx.set_xlabel("Samples")
        bx.set_title("Waveforms")
        plt.show()
        break




In [ ]:
event.r1.tel[tel_numbers[i]].waveform.shape

In [ ]:
def test_func():
    run = 1
    tel_number = 1
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{run}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    for i in range(1,100):
        event_iterator = iter(source)
        event = next(event_iterator)
        pixel_mask = []
        for tels in event.trigger.tels_with_trigger:
            for k,l in enumerate(event.r0.tel[tels].waveform[1]):
                if np.argmax(l) in range(16,40): 
                    pixel_mask.append(True)
                else:
                    pixel_mask.append(False)

            working_wave = event.r1.tel[tels].waveform.copy()
            for k,l in enumerate(working_wave):
                if pixel_mask[k]:
                    #working_wave[k] = np.zeros(40)     
                    working_wave[k] = np.mean(event.r1.tel[tels].waveform, axis = 0)


            event.r1.tel[tels].waveform = working_wave

        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray)
        calib(event)
        image_processor(event)
        muon_processor(event)

In [ ]:
type(event.r0.tel[tel_number].waveform[1])

In [ ]:
test_func()

In [ ]:
filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{run}.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)
event = next(event_iterator)
print(type(event))
print(event.r0.tel[3].waveform[1])

In [ ]:
print(event.trigger.tels_with_trigger)

In [ ]:
event.muon.tel[1].parameters.intensity_ratio

In [ ]:
x_range = [i for i in range(10)]
#std_arr = [np.std(opt_eff) for i,j in enumerate(opt_eff)]
y_upper = [0.1749+0.0258 for i in x_range]
y_lower = [0.1749-0.0258 for i in x_range]
plt.xlabel("Event")
#plt.ylabel("Optical eff with rel. error for 1 measurment")
plt.ylabel("Optical eff with std for 1 measurment")
plt.title("Opt eff for muon rings without performing additional cleaning int_cut<0.15")
plt.grid(alpha=0.5)
#plt.ylim(0,0.4)
plt.fill_between(x_range, y_upper, y_lower, color = 'g', alpha = 0.3, label = 'mean +/- 1 std')

In [ ]:
check_mean = np.mean(event.r1.tel[tel_number].waveform, axis = 0)
check_mean[13]

In [ ]:
event.r1.tel[tel_number].waveform

In [ ]:
###
###
### EXTRACT SHOWER FROM IMAGE FOR R0 R0 R0 R0 R0 R0 R0 R0 R0 R0 R0 R0 R0 R0 R0 R0 R0 R0 R0 R0 R0
### FOR TESTS
###
###


#
#
#
#
#
# Main algorihtm to process events with mask to extract ONLY muon ring
#
#
#
#
#

deleted_pixels = []

#2223, 9222, 2438, 25051, 14406, 22126, 22365, 32298, 35219, 8234, 8448, 9404, 9433, 27374, 28328, 28000, 35298
contaminated_runs = [2,9,2,25,14,22,22, 32,35,8,8,9,9,27,28,28,35]
contaminated_events = [223, 222, 438, 51, 406, 126, 365, 298, 219, 234, 448, 404, 433, 374, 328, 0, 298]
tel_numbers = [2, 2,3,1,1,1,1, 3, 2, 1,1,1,3,2,1,1,2]
events = []
working_waves = []


calib_config = Config({"GlobalPeakWindowSum": {"window_width" :7, "window_shift" : 3}})



for i,j in enumerate(contaminated_runs):
    counter = 0
    tel_number = tel_numbers[i]
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{j}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    for k,l in enumerate(event_iterator):
        if k == contaminated_events[i]:
            event = l
            events.append(event)
            break
    # Building appropriate image for this event
    
    
    fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
    fig.subplots_adjust(hspace=0.2)
    ax = axes[0]
    camgeom = source.subarray.tel[1].camera.geometry
    title= f"True Image of Event_{str(contaminated_runs[i])+'run'+str(contaminated_events[i])}"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax = ax)
    #disp.image = event.simulation.tel[tel_number].true_image
    disp.image = event.r1.tel[tel_number].waveform.sum(axis=1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    bx = axes[1]
    pixel_mask = []
    for k,l in enumerate(event.r0.tel[tel_number].waveform[1]):
        if np.argmax(l) in range(16,40): 
            pixel_mask.append(True)
            counter += 1
        else:
            pixel_mask.append(False)
    working_wave = event.r1.tel[tel_number].waveform.copy()
    for k,l in enumerate(working_wave):
        if pixel_mask[k]:
            working_wave[k] = np.zeros(40)
            #working_wave[k] = np.min(event.r1.tel[tel_number].waveform, axis = 0)     
            #working_wave[k] = np.mean(event.r1.tel[tel_number].waveform, axis = 0)
            #working_wave[k] = np.min(event.r0.tel[tel_number].waveform[0], axis = 0)
    working_waves.append(working_wave)
    temp = np.mean(event.r1.tel[tel_number].waveform, axis = 0)
    #print("mean value before = ", np.mean(event.r1.tel[tel_number].waveform, axis = 0))
    event.r1.tel[tel_numbers[i]].waveform = working_wave
    #print("mean value after = ", np.mean(event.r1.tel[tel_number].waveform, axis = 0))
    #print("delta before-after = ", (temp - np.mean(event.r1.tel[tel_number].waveform, axis = 0)) / np.mean(event.r1.tel[tel_number].waveform, axis = 0) *100 )
    #print("mean value = ", np.mean(event.r1.tel[tel_number].waveform, axis = 0))
    #print("min and max = ", np.min(event.r1.tel[tel_number].waveform, axis = 0), np.max(event.r1.tel[tel_number].waveform, axis = 0))
    image_processor = ImageProcessor(source.subarray)
    muon_processor = MuonProcessor(source.subarray)
    calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray, config = calib_config)
    calib(event)
    image_processor(event)
    muon_processor(event)
    deleted_pixels.append(counter)
    #opt_effi = []
    #if not np.isnan(event.muon.tel[tel_numbers[i]].efficiency.optical_efficiency):
    #opt_effi.append(event.muon.tel[tel_numbers[i]])
    camgeom = source.subarray.tel[1].camera.geometry
    #title= f"Extracted by mask. fitted optical eff = {round(event.muon.tel[tel_number].efficiency.optical_efficiency, 3)}"
    title = "Extracted shower"
    disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True, ax=bx)
    disp.image = working_wave.sum(axis=1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    plt.show()
    #plt.savefig(f"/Users/vdk/ExtractedContamination<16timebinMEAN_event{str(contaminated_runs[i])+str(contaminated_events[i])}")
    
plt.close()





In [ ]:
arr = np.array([0.182, 0.198, 0.198, 0.231, 0.202, 0.205, 0.3])
np.mean(arr),np.std(arr)

In [ ]:
np.std(arr)/np.mean(arr)

In [ ]:
bad_pix = np.array([0,1])

arr = arr[~bad_pix]

In [ ]:
print(event.dl1.tel[1].image)

In [ ]:
event.dl1.tel[1].image = event.dl1.tel[1].image[~bad_pix]

In [ ]:
len(event.dl1.tel[1].image)

In [ ]:
event.dl1.tel[1].image

In [ ]:
array1 = np.array([1,2,3,4])
bad = np.array([0,1])
print(array1[~bad])

In [ ]:
40/3*40